In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

In [2]:
df_rating = pd.read_csv('data/mbti/ratings.csv')
df_personality = pd.read_csv('data/mbti/personality.csv')
df = pd.merge(df_rating, df_personality, on='user_id')

mbti_types = ['INTJ', 'INTP', 'ENTJ', 'ENTP', 'INFJ', 'INFP', 'ENFJ', 'ENFP', 'ISTJ', 'ISFJ', 'ESTJ', 'ESFJ', 'ISTP', 'ISFP', 'ESTP', 'ESFP']

In [3]:
# Define the number of unique users, movies, and personality dimensions
num_users = df['user_id'].nunique()
num_movies = df['movie_id'].nunique()
num_mbti_types = 16

In [4]:
# Define the embedding dimensions
embedding_dim = 16

# Define the input layers
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))
mbti_input = Input(shape=(num_mbti_types,))

# Create user and movie embeddings
user_embedding = Embedding(num_users, embedding_dim)(user_input)
user_embedding = Flatten()(user_embedding)

movie_embedding = Embedding(num_movies, embedding_dim)(movie_input)
movie_embedding = Flatten()(movie_embedding)

# Concatenate user, movie, and MBTI embeddings
concat = Concatenate()([user_embedding, movie_embedding, mbti_input])

# Create a 4-layer MLP
layer1 = Dense(64, activation='relu')(concat)
layer2 = Dense(32, activation='relu')(layer1)
layer3 = Dense(16, activation='relu')(layer2)
output = Dense(1)(layer3)  # Output layer for rating prediction

# Create the model
model = Model(inputs=[user_input, movie_input, mbti_input], outputs=output)

In [5]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [12]:
# Train the model for 10 epochs with a batch size of 64
df_train = pd.read_csv('data/mbti/ratings-train.csv')

train_user_ids = df_train['user_id'].values
train_movie_ids = df_train['movie_id'].values
train_ratings = df_train['rating'].values
train_mbti_scores = df_train[mbti_types].values


In [13]:
# Train the model
model.fit([train_user_ids, train_movie_ids, train_mbti_scores], train_ratings, epochs=10, batch_size=64)

Epoch 1/10
13375/13375 [==============================] - 104s 8ms/step - loss: 0.7115
Epoch 2/10
13375/13375 [==============================] - 122s 9ms/step - loss: 0.6065
Epoch 3/10
13375/13375 [==============================] - 113s 8ms/step - loss: 0.5733
Epoch 4/10
13375/13375 [==============================] - 99s 7ms/step - loss: 0.5463
Epoch 5/10
13375/13375 [==============================] - 102s 8ms/step - loss: 0.5225
Epoch 6/10
13375/13375 [==============================] - 112s 8ms/step - loss: 0.5014
Epoch 7/10
13375/13375 [==============================] - 102s 8ms/step - loss: 0.4837
Epoch 8/10
13375/13375 [==============================] - 112s 8ms/step - loss: 0.4687
Epoch 9/10
13375/13375 [==============================] - 113s 8ms/step - loss: 0.4560
Epoch 10/10
13375/13375 [==============================] - 112s 8ms/step - loss: 0.4448


In [15]:
model.save('saved-models/ncf-most-salient-personality-mbti.h5')